# Hugging Face Sentiment Classification
__Binary Classification with `Trainer` and `sst2` dataset__

## Runtime

This notebook takes approximately 45 minutes to run.

## Contents

1. [Introduction](#Introduction)  
2. [Development environment and permissions](#Development-environment-and-permissions)
    1. [Installation](#Installation)  
    2. [Development environment](#Development-environment)  
    3. [Permissions](#Permissions)
3. [Pre-processing](#Pre-processing)   
    1. [Tokenize sentences](#Tokenize-sentences)  
    2. [Upload data to sagemaker_session_bucket](#Upload-data-to-sagemaker_session_bucket)  
4. [Fine-tune the model and start a SageMaker training job](#Fine-tune-the-model-and-start-a-SageMaker-training-job)  
    1. [Create an Estimator and start a training job](#Create-an-Estimator-and-start-a-training-job)  
    2. [Estimator Parameters](#Estimator-Parameters)   
    3. [Attach a previous training job to an estimator](#Attach-a-previous-training-job-to-an-estimator)  

## Introduction

Welcome to our end-to-end binary text classification example. This notebook uses Hugging Face's `transformers` library with a custom Amazon sagemaker-sdk extension to fine-tune a pre-trained transformer on binary text classification. The pre-trained model is fine-tuned using the `sst2` dataset. To get started, we need to set up the environment with a few prerequisite steps for permissions, configurations, and so on. 

This notebook is adapted from Hugging Face's notebook [Huggingface Sagemaker-sdk - Getting Started Demo](https://github.com/huggingface/notebooks/blob/master/sagemaker/01_getting_started_pytorch/sagemaker-notebook.ipynb) and provided here courtesy of Hugging Face.

<img src="text_classification.png" width="700"/>

## Runtime

This notebook takes approximately 40 minutes to run.

<i>NOTE: You can run this notebook in SageMaker Studio, a SageMaker notebook instance, or your local machine. This notebook was tested in a notebook instance using the conda\_pytorch\_p36 kernel.</i>


## Development environment and permissions 

### Installation

_*Note:* We install the required libraries from Hugging Face and AWS. You also need PyTorch, if you haven't installed it already._

In [1]:
!pip install "sagemaker" "transformers" "datasets[s3]" "s3fs" --upgrade

  Using cached sagemaker-2.103.0-py2.py3-none-any.whl
     |████████████████████████████████| 4.0 MB 4.7 MB/s            
     |████████████████████████████████| 365 kB 75.0 MB/s            
     |████████████████████████████████| 880 kB 73.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 67 kB 8.2 MB/s             
     |████████████████████████████████| 6.6 MB 68.7 MB/s            
     |████████████████████████████████| 211 kB 72.3 MB/s            
     |████████████████████████████████| 133 kB 69.8 MB/s            
     |████████████████████████████████| 78 kB 10.1 MB/s            
     |████████████████████████████████| 58 kB 8.5 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 8.4 MB 32.5 MB/s            
     |████████████████████████████████| 132 kB 75.1 MB/s            
     |████████████████████████████████| 132 kB 72.1 MB/s            
     |████████████████████████████████| 

     |████████████████████████████████| 132 kB 36.3 MB/s            
     |████████████████████████████████| 132 kB 71.9 MB/s            
     |████████████████████████████████| 132 kB 74.8 MB/s            
     |████████████████████████████████| 132 kB 72.2 MB/s            
     |████████████████████████████████| 132 kB 73.5 MB/s            
     |████████████████████████████████| 132 kB 74.0 MB/s            
     |████████████████████████████████| 132 kB 72.8 MB/s            
     |████████████████████████████████| 132 kB 69.7 MB/s            
     |████████████████████████████████| 132 kB 73.5 MB/s            
     |████████████████████████████████| 132 kB 70.6 MB/s            
     |████████████████████████████████| 132 kB 73.6 MB/s            
     |████████████████████████████████| 132 kB 75.0 MB/s            
     |████████████████████████████████| 132 kB 71.5 MB/s            
     |████████████████████████████████| 132 kB 58.5 MB/s            
     |████████████████████████████

     |████████████████████████████████| 132 kB 74.0 MB/s            
     |████████████████████████████████| 132 kB 67.9 MB/s            
     |████████████████████████████████| 132 kB 73.7 MB/s            
     |████████████████████████████████| 132 kB 67.8 MB/s            
     |████████████████████████████████| 132 kB 71.8 MB/s            
     |████████████████████████████████| 131 kB 70.0 MB/s            
     |████████████████████████████████| 131 kB 71.9 MB/s            
     |████████████████████████████████| 131 kB 74.1 MB/s            
     |████████████████████████████████| 131 kB 73.6 MB/s            
     |████████████████████████████████| 131 kB 74.7 MB/s            
     |████████████████████████████████| 131 kB 72.1 MB/s            
     |████████████████████████████████| 131 kB 74.9 MB/s            
     |████████████████████████████████| 131 kB 72.7 MB/s            
     |████████████████████████████████| 131 kB 70.4 MB/s            
     |████████████████████████████

  Attempting uninstall: boto3
    Found existing installation: boto3 1.23.10
    Uninstalling boto3-1.23.10:
      Successfully uninstalled boto3-1.23.10
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.102.0
    Uninstalling sagemaker-2.102.0:
      Successfully uninstalled sagemaker-2.102.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.
spacy 3.0.6 requires pydantic<1.8.0,>=1.7.1, but you have pydantic 1.8.2 which is incompatible.
awscli 1.24.10 requires botocore==1.26.10, but you have botocore 1.23.24 which is incompatible.


### Development environment 

In [2]:
import sagemaker.huggingface

### Permissions

_If you are going to use SageMaker in a local environment, you need access to an IAM Role with the required permissions for SageMaker. You can read more at [SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)._

In [3]:
import sagemaker

sess = sagemaker.Session()
# The SageMaker session bucket is used for uploading data, models and logs
# SageMaker will automatically create this bucket if it doesn't exist
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # Set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"Role arn: {role}")
print(f"Bucket: {sess.default_bucket()}")
print(f"Region: {sess.boto_region_name}")

Role arn: arn:aws:iam::485013493900:role/UserSageMakerRole
Bucket: sagemaker-us-east-2-485013493900
Region: us-east-2


## Pre-processing

We use the `datasets` library to pre-process the `sst2` dataset (Stanford Sentiment Treebank). After pre-processing, the dataset is uploaded to the `sagemaker_session_bucket` for use within the training job. The [sst2](https://nlp.stanford.edu/sentiment/index.html) dataset consists of 67349 training samples and _ testing samples of highly polar movie reviews.

### Download the dataset

In [4]:
from datasets import Dataset
from transformers import AutoTokenizer
import pandas as pd

# Tokenizer used in pre-processing
tokenizer_name = "distilbert-base-uncased"

# S3 key prefix for the data
s3_prefix = "DEMO-samples/datasets/sst"

# Download the SST2 data from s3
!curl https://sagemaker-sample-files.s3.amazonaws.com/datasets/text/SST2/sst2.test > ./sst2.test
!curl https://sagemaker-sample-files.s3.amazonaws.com/datasets/text/SST2/sst2.train > ./sst2.train
!curl https://sagemaker-sample-files.s3.amazonaws.com/datasets/text/SST2/sst2.val > ./sst2.val

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189k  100  189k    0     0  1308k      0 --:--:-- --:--:-- --:--:-- 1312k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3716k  100 3716k    0     0  20.2M      0 --:--:-- --:--:-- --:--:-- 20.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94916  100 94916    0     0   907k      0 --:--:-- --:--:-- --:--:--  908k


### Tokenize sentences

In [5]:
# Download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Tokenizer helper function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)


# Load dataset
test_df = pd.read_csv("sst2.test", sep="delimiter", header=None, engine="python", names=["line"])
train_df = pd.read_csv("sst2.train", sep="delimiter", header=None, engine="python", names=["line"])

test_df[["label", "text"]] = test_df["line"].str.split(" ", 1, expand=True)
train_df[["label", "text"]] = train_df["line"].str.split(" ", 1, expand=True)

test_df.drop("line", axis=1, inplace=True)
train_df.drop("line", axis=1, inplace=True)

test_df["label"] = pd.to_numeric(test_df["label"], downcast="integer")
train_df["label"] = pd.to_numeric(train_df["label"], downcast="integer")

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Set format for pytorch
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

### Upload data to `sagemaker_session_bucket`

After processing the `datasets`, we use the `FileSystem` [integration](https://huggingface.co/docs/datasets/filesystems.html) to upload the dataset to S3.

In [6]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()

# save train_dataset to s3
training_input_path = f"s3://{sess.default_bucket()}/{s3_prefix}/train"
train_dataset.save_to_disk(training_input_path, fs=s3)

# save test_dataset to s3
test_input_path = f"s3://{sess.default_bucket()}/{s3_prefix}/test"
test_dataset.save_to_disk(test_input_path, fs=s3)

In [7]:
print("training_input_path:", training_input_path)
print("test_input_path:", test_input_path)


training_input_path: s3://sagemaker-us-east-2-485013493900/DEMO-samples/datasets/sst/train
test_input_path: s3://sagemaker-us-east-2-485013493900/DEMO-samples/datasets/sst/test


## Fine-tune the model and start a SageMaker training job

In order to create a SageMaker training job, we need a `HuggingFace` Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. In an Estimator, we define which fine-tuning script should be used as `entry_point`, which `instance_type` should be used, which `hyperparameters` are passed in, etc:



```python
hf_estimator = HuggingFace(entry_point="train.py",
                            source_dir="./scripts",
                            base_job_name="huggingface-sdk-extension",
                            instance_type="ml.p3.2xlarge",
                            instance_count=1,
                            transformers_version="4.4",
                            pytorch_version="1.6",
                            py_version="py36",
                            role=role,
                            hyperparameters = {"epochs": 1,
                                               "train_batch_size": 32,
                                               "model_name":"distilbert-base-uncased"
                                                })
```

When we create a SageMaker training job, SageMaker takes care of starting and managing all the required EC2 instances for us with the `huggingface` container, uploads the provided fine-tuning script `train.py`, and downloads the data from the `sagemaker_session_bucket` into the container at `/opt/ml/input/data`. Then, it starts the training job by running:

```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
```

The `hyperparameters` defined in the `HuggingFace` estimator are passed in as named arguments. 

SageMaker provides useful properties about the training environment through various environment variables, including the following:

* `SM_MODEL_DIR`: A string representing the path where the training job writes the model artifacts to. After training, artifacts in this directory are uploaded to S3 for model hosting.

* `SM_NUM_GPUS`: An integer representing the number of GPUs available to the host.

* `SM_CHANNEL_XXXX:` A string representing the path to the directory that contains the input data for the specified channel. For example, if you specify two input channels in the Hugging Face estimator's `fit()` call, named `train` and `test`, the environment variables `SM_CHANNEL_TRAIN` and `SM_CHANNEL_TEST` are set.


To run the training job locally, you can define `instance_type="local"` or `instance_type="local_gpu"` for GPU usage.

_Note: local mode is not supported in SageMaker Studio._


In [8]:
!pygmentize ./scripts/train.py

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_from_disk
import random
import logging
import sys
import argparse
import os
import torch

if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # Hyperparameters sent by the client are passed as command-line arguments to the script
    parser.add_argument("--epochs", type=int, default=3)
    parser.add_argument("--train_batch_size", type=int, default=32)
    parser.add_argument("--eval_batch_size", type=int, default=64)
    parser.add_argument("--warmup_steps", type=int, default=500)
    parser.add_argument("--model_name", type=str)
    parser.add_argument("--learning_rate", type=str, default=5e-5)

    # Data, model, and output directories
    parser.add_argument("--output_data_dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--mode

### Create an Estimator and start a training job

In [9]:
from sagemaker.huggingface import HuggingFace

# Hyperparameters which are passed into the training job
hyperparameters = {"epochs": 1, "train_batch_size": 32, "model_name": "distilbert-base-uncased"}

In [10]:
hf_estimator = HuggingFace(
    entry_point="train.py",
    source_dir="./scripts",
    instance_type="ml.p3.2xlarge",
    instance_count=1,
    role=role,
    transformers_version="4.12",
    pytorch_version="1.9",
    py_version="py38",
    hyperparameters=hyperparameters,
)

In [11]:
# Start the training job with the uploaded dataset as input
hf_estimator.fit({"train": training_input_path, "test": test_input_path})

2022-08-08 15:11:13 Starting - Starting the training job...
2022-08-08 15:11:40 Starting - Preparing the instances for trainingProfilerReport-1659971473: InProgress
.........
2022-08-08 15:12:59 Downloading - Downloading input data...
2022-08-08 15:13:37 Training - Downloading the training image..............................
2022-08-08 15:18:38 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-08-08 15:18:36,617 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-08-08 15:18:36,640 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-08-08 15:18:36,648 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-08-08 15:18:37,304 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 67349
***** Running training *****
  Num examples = 67349
Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
Gradient Accumulation steps = 1
  Total optimization steps = 2105
Gradient Accumulation steps = 1
  Total optimization steps = 2105
0%|          | 0/2105 [00:00<?, ?it/s]
[2022-08-08 15:18:54.341 algo-1:26 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-08-08 15:18:54.463 algo-1:26 INFO profiler_config_parser

8%|▊         | 161/2105 [01:19<15:33,  2.08it/s]


15%|█▍        | 307/2105 [02:29<14:20,  2.09it/s]


22%|██▏       | 463/2105 [03:44<13:10,  2.08it/s]


...

82%|████████▏ | 1735/2105 [14:00<02:57,  2.08it/s]


89%|████████▉ | 1881/2105 [15:10<01:48,  2.07it/s]


95%|█████████▌| 2000/2105 [16:08<00:50,  2.10it/s]
{'loss': 0.1648, 'learning_rate': 3.2710280373831774e-06, 'epoch': 0.95}
95%|█████████▌| 2000/2105 [16:08<00:50,  2.10it/s]
Saving model checkpoint to /opt/ml/model/checkpoint-2000
Saving model checkpoint to /opt/ml/model/checkpoint-2000
Configuration saved in /opt/ml/model/checkpoint-2000/config.json
Configuration saved in /opt/ml/model/checkpoint-2000/config.json
Model weights saved in /opt/ml/model/checkpoint-2000/pytorch_model.bin
Model weights saved in /opt/ml/model/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in /opt/ml/model/checkpoint-2000/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/checkpoint-2000/tokenizer_config.json
Special tokens file saved in /opt/ml/model/checkpoint-2000/special_tokens_map.json
Special tokens file saved in /opt/ml/model/checkpoint-2000/special_tokens_map.json
96%|█████████▌| 2013/2105 [16:16<00:44,  2.05it/s]


100%|██████████| 2105/2105 [16:59<00:00,  2.31it/s]
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
***** Running Evaluation *****
  Num examples = 1821
Num examples = 1821
  Batch size = 64
Batch size = 64
100%|██████████| 29/29 [00:09<00:00,  3.57it/s]#033[A
#033[A
{'eval_loss': 0.20894648134708405, 'eval_accuracy': 0.9187259747391543, 'eval_f1': 0.920855614973262, 'eval_precision': 0.8959417273673257, 'eval_recall': 0.9471947194719472, 'eval_runtime': 9.6277, 'eval_samples_per_second': 189.143, 'eval_steps_per_second': 3.012, 'epoch': 1.0}
#015100%|██████████| 29/29 [00:09<00:00,  3.57it/s]
#033[A
#033[A
Training completed. Do not forget to share your model on huggingface.co/models =)
Tr

100%|██████████| 29/29 [00:09<00:00,  3.17it/s]
***** Eval results *****
Saving model checkpoint to /opt/ml/model
Saving model checkpoint to /opt/ml/model
Configuration saved in /opt/ml/model/config.json
Configuration saved in /opt/ml/model/config.json
Model weights saved in /opt/ml/model/pytorch_model.bin
Model weights saved in /opt/ml/model/pytorch_model.bin
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
tokenizer config file saved in /opt/ml/model/tokenizer_config.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
Special tokens file saved in /opt/ml/model/special_tokens_map.json
2022-08-08 15:36:14,630 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2022-08-08 15:36:14,631 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2022-08-08 15:36:14,631 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2022-08-08 15:36:44 Uploading - Upload

### Deploy the endpoint

To deploy the endpoint, call `deploy()` on the HuggingFace estimator object, passing in the desired number of instances and instance type.

In [ ]:
predictor = hf_estimator.deploy(1, "ml.p3.2xlarge")

------------

Then use the returned predictor object to perform inference.

In [ ]:
sentiment_input = {"inputs": "I love using the new Inference DLC."}

predictor.predict(sentiment_input)

We see that the fine-tuned model classifies the test sentence "I love using the new Inference DLC." as having positive sentiment with 98% probability!

Finally, delete the endpoint.

In [ ]:
predictor.delete_endpoint()

## Extras

### Estimator Parameters

In [ ]:
print(f"Container image used for training job: \n{hf_estimator.image_uri}\n")
print(f"S3 URI where the trained model is located: \n{hf_estimator.model_data}\n")
print(f"Latest training job name for this estimator: \n{hf_estimator.latest_training_job.name}\n")

In [ ]:
hf_estimator.sagemaker_session.logs_for_job(hf_estimator.latest_training_job.name)

### Attach a previous training job to an estimator

In SageMaker, you can attach a previous training job to an estimator to continue training, get results, etc.

In [ ]:
from sagemaker.estimator import Estimator

# Uncomment the following lines and supply your training job name

# old_training_job_name = "<your-training-job-name>"
# hf_estimator_loaded = Estimator.attach(old_training_job_name)
# hf_estimator_loaded.model_data